In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import konlpy 
from tkinter import *
from tkinter import ttk
from tkinter import messagebox


df = pd.read_json('투자 유치 100억이상 회사 복지 혜택.json')

# 결측치를 채워주는 부분
df = df.fillna('')
df_columns = df.columns.values
df_columns= np.delete(df_columns,[0])

In [2]:
# 데이터 프레임을 받아와서 특정 행값을 특정 열에 대해서 유사도를 도출해내는 함수

def tfidf_vect_func(df, target_col, target_row, tokenizer):

    # 특정 열을 형태소로 만들어준다. 왜 만들어주느냐 성능을 높이기 위해서
    col_tokenized = [ tokenizer.morphs(item) for item in df[target_col] ] 
    col_tokenized = [ ' '.join(item) for item in col_tokenized ]

    # 문자열 데이터를 tfid를 통해서 숫자형 데이터로 바꿔주는 부분
    tfidf_vect = TfidfVectorizer()
    feature_vect = tfidf_vect.fit_transform(col_tokenized)

    # 유사도를 도출해내는 부분
    similarity_simple_pair = cosine_similarity(feature_vect[target_row], feature_vect)
    result_list = similarity_simple_pair.tolist()[0]
    df[f'{target_col}_result'] = result_list
    
    return df

In [3]:
choice = input('회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : ')

회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : 0


In [4]:
a = df['Company_name'].values.tolist()

#콤보박스 선택 값 돌려주기
def change1(index, value, op):
    print(brand_combo.get())
#     print(brand_combo.current())

    brand_index = brand_combo.current()
    label = ttk.Label(win, text=brand_index)
    label.grid(row = 0, column=1)

brand = a

# window 띄우기
win = Tk ()
win.title("스타트업 회사 선택")
win.geometry('400x200')
win.resizable(FALSE, FALSE)

# brand 콤보박스
str1 = StringVar()
str1.trace('w', change1)
brand_combo = ttk.Combobox(win, width=20, state='readonly', textvariable=str1, values=brand)
brand_combo.grid(row = 0, column=0)

win.bind('<Escape>', lambda e: win.destroy()) # ESC 버튼 누르면 창 닫기

win.mainloop()

클룩


In [7]:
%%capture cap --no-stderr 
global input_company

input_company = cap.stdout.strip() # cap 객체에 저장된 값을 test 변수에 할당
print(input_company)

In [9]:
input_company = input()

야놀자


In [10]:
df[df['Company_name'] == input_company]

,Company_name,Work type,"meals, snacks","annual leave, vacation","insurance, medical service",equipment,self development,transportaion
17,야놀자,자율 출근,"월1회 사내이벤트, 고급 커피 제공, 중식 제공","연차 휴가, 출산 휴가, 월차","의료 실비 보험, 경조사비, 종합 검진 제공, 4대 보험","듀얼모니터(델), iMac 27"", 맥북 프로 13"", 맥북 프로 15""","생일축하금, 명절 선물 지급, 명절상여금, 탁구장, 베드민턴장, 도서 구입비 지원,...",야근교통비지원


In [18]:
choice = input('회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : ')

if choice == '0' : 
    print(input_company)
#     input_company = input("가장 마음에 드는 회사를 선택하세요. : ")
    target_row= df[df['Company_name'] == input_company].index[0]


    okt = konlpy.tag.Okt()

# 데이터프레임에 유사도 추가
    for num, target_col in enumerate(df_columns):
        print('cosine_similarity 분석 중.. '+ str(num+1) + '/'+ str(len(df_columns)))
        tfidf_vect_func(df, target_col,target_row, okt)

    result_arr = []
# 유사도 열만 가져오기
    for column in df.columns.values :
        if 'result' in column:
            result_arr.append(column)


    df_result = df[result_arr]
    df_result_weight = df_result.copy()


    print()
    print ("근무 형태 : 0, 식사, 간식 : 1, 연차, 휴가 : 2 ,보험, 의료: 3 개인 장비: 5, 자기 계발: 5, 통근, 교통: 6")

    weight1 = input("가장 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")  #  가장 중요하게 생각하는 회사의 복지
    weight2 = input("두 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")
    weight3 = input("세 번쨰로 중요시하는 회사의 복지를 숫자의 형태로 입력해주세요. : ")

# 가중치 추가
    df_result_weight.iloc[:,int(weight1)] = df_result_weight.iloc[:, int(weight1)] * 1.75
    df_result_weight.iloc[:,int(weight2)] = df_result_weight.iloc[:,int(weight2)] * 1.5
    df_result_weight.iloc[:,int(weight3)] = df_result_weight.iloc[:,int(weight3)] * 1.25

    df['sum'] = df_result_weight.cumsum(axis=1).iloc[:,-1]

# 유사도의 합 기준으로 정렬
    df_result_sort = df.sort_values(by='sum', ascending=False).copy()

    # print_df(df_result_sort.head(10))
    result_df = df_result_sort.head(10)

        
elif choice == '1' :
    
# 모든 열을 하나의 열로 만들어 줌
    df['mess'] = df.iloc[:,1:].sum(axis=1)
    df_columns = df.columns.values[1:]
    df = df.append(pd.Series(),ignore_index= True)
    last_index = df.iloc[[-1]].index[0]

    df.at[last_index,'mess'] =input('원하시는 복지를 입력해주세요.')



    okt = konlpy.tag.Okt()

    # for target_col in df_columns:
    #     tfidf_vect_func(df, target_col,target_row, okt)

# 유사도 계산
    tfidf_vect_func(df,'mess',last_index,okt)
    
# 유사도 기준으로 정렬
    df_result_sort = df.sort_values(by='mess_result', ascending=False).copy()
    df_result_sort_mess = df_result_sort.drop(columns='mess')

    # print_df(df_result_sort_mess.iloc[1:])
    result_df = df_result_sort_mess.iloc[1:]

else :
    # print('0 혹은 1 을 입력하세요.')
    result_df = '0 혹은 1 을 입력하세요.'
    

result_df['recruit'] = 'https://www.wanted.co.kr/search?query=' + result_df['Company_name']

# url열에 추가

def make_clickable(link):
    text = link.split('=')[1]
    return f'<a target="_blank" href="{link}">{text}</a>'

result_df = result_df.style.format({'recruit': make_clickable})    
    
result_df

야놀자
회사를 입력하시려면 0, 복지를 입력하시려면 1 을 입력하세요. : 1


C:\Users\jangyunsik\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
C:\Users\jangyunsik\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


KeyboardInterrupt: Interrupted by user